In [319]:
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

from numpy import genfromtxt




# # convert ndarray to tensor
def to_tensor(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  sess=tf.InteractiveSession()
  print arg.eval()
  return arg

# Load in training and testing data
# Shape of (record_num, col_num): (999,7)
my_data = genfromtxt('Data/modelData.csv', delimiter=',')
my_testing = genfromtxt('Data/testData.csv', delimiter=',')

category = 5
factor = 1
sim = 0.1



In [320]:


# batch function
def my_batch(category,num,my_data):
#     choose a num of batch return x, y as vectors
    size,col = my_data.shape
    idList = random.sample(range(0, size), num )
    batch_xs = np.zeros(shape=(num,col-category))
    batch_ys = np.zeros(shape=(num,category))
    
    for index,i in enumerate(idList):
        line = my_data[i,:-category]
        batch_xs[index]=line
        label = my_data[i,col-category:col]
        batch_ys[index]=label
    return batch_xs,batch_ys
    
def testingDataLoader(category,my_data):
    size,col = my_data.shape
    
    batch_xs = np.zeros(shape=(size,col-category))
    batch_ys = np.zeros(shape=(size,category))
    for index,line in enumerate(my_data):
        batch_xs[index]=line[:-category]
        batch_ys[index]=line[col-category:col]

        
    return batch_xs,batch_ys

In [321]:
# def compare(x, y, name=None):
# #     sess=tf.InteractiveSession()
# #     sess.run(x)
# #     sess.run(y)  
#     a = tf.reshape(x,[-1]).eval()
#     b = tf.reshape(y,[-1]).eval()
# #       print a
# #       print b
#     sum = abs(np.dot(a,b))
#     cos = sum/(LA.norm(a)*LA.norm(b))
# #        print "cos",cos
#     return to_tensor(cos)

def compare(x,y,name=None):
#     sess=tf.InteractiveSession()

    norm1= tf.sqrt(tf.reduce_sum(tf.square(x),keep_dims=True))
    norm2= tf.sqrt(tf.reduce_sum(tf.square(y),keep_dims=True))
    dot = tf.reduce_sum(tf.mul(x,y))
#     print "1",norm1
#     print "2",norm2
#     print dot
    cos = abs(dot)/tf.mul(norm1,norm2)
    
#     print "cos",type(cos)
#     print "change",type(to_tensor(cos[0]))
#     return tf.convert_to_tensor(cos_, dtype=tf.float32)
    return cos


In [322]:
# Implementation starts!

# None means any number, so x is not a specific number here.
x = tf.placeholder(tf.float32, [None, factor],name="x-input")


# In[16]:

# Init weights, bias, (all zeros first) and define softmax function
W = tf.Variable(tf.zeros([factor, category]),name="weights")
b = tf.Variable(tf.zeros([category],name="bias"))

# use a name scope to organize nodes in the graph visualizer
with tf.name_scope("Wx_b") as scope:
# first multiply x and w, then add b vector. apply softmax to get probabilities
  y = tf.nn.softmax(tf.matmul(x, W) + b)
#     y = tf.matmul(x, W) + b

# Add summary ops to collect data
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)


weights_summary = tf.histogram_summary("weights", W)
biases_summary = tf.histogram_summary("biases", b)
y_summary = tf.histogram_summary("y", y)


# In[20]:

# Trainning
y_ = tf.placeholder(tf.float32, [None, category],name="y-input")

with tf.name_scope("xent") as scope:
# tf.log computes logarithm of each element. 
  cross_entropy = -tf.reduce_sum(y_*tf.log(y))
    
with tf.name_scope("train") as scope:
# minimize cross_entropy using the gradient descent algorithm with a learning rate of 0.01. 
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


# In[21]:



In [323]:


# In[22]:


  


  
# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.merge_summary([weights_summary, biases_summary, y_summary])
writer = tf.train.SummaryWriter("/tmp/read2", sess.graph_def)
tf.train.SummaryWriter("/tmp/read2", sess.graph_def).flush()
# tf.train.SummaryWriter.flush()



In [324]:

with tf.name_scope("test") as scope:
# evaluation
#   correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))   -> [T,F,T,T...]
#   accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    norm1= tf.sqrt(tf.reduce_sum(tf.square(y),keep_dims=True))
    norm2= tf.sqrt(tf.reduce_sum(tf.square(y_),keep_dims=True))
    dot = tf.reduce_sum(tf.mul(y,y_))
    accuracy = tf.reduce_sum(abs(dot)/tf.mul(norm1,norm2))
    accuracy = tf.reduce_sum(y)
    accuracy_summary = tf.scalar_summary("accuracy", accuracy)



In [325]:



  
init = tf.initialize_all_variables()

# launch the model in a Session, run the initialized operation
sess = tf.Session()
sess.run(init)





# get testing data
test_xs,test_ys = testingDataLoader(category,my_testing)


# Train for 1000 times!
# batch of 100 at each time
# train_step feeding in the batches data to replace the placeholders
for i in range(10000):


  batch_xs, batch_ys = my_batch(category,100, my_data)
  feed = {x: batch_xs, y_: batch_ys}

  if i%100 == 0:
    train_accuracy = accuracy.eval(session=sess, feed_dict=feed)
#     print "type ",tf.shape(accuracy)
    print "value ", train_accuracy
    print "step %d, training accuracy %g"%(i, train_accuracy)
    train_step.run(session=sess, feed_dict=feed)


#     batch_xs, batch_ys = my_batch(category,100, my_data)
#     feed = {x: batch_xs, y_: batch_ys}
#     sess.run(train_step, feed_dict=feed)




type  Tensor("Shape_2192:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  99.9996
step 0, training accuracy 99.9996
type  Tensor("Shape_2193:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  100.0
step 100, training accuracy 100
type  Tensor("Shape_2194:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 200, training accuracy nan
type  Tensor("Shape_2195:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 300, training accuracy nan
type  Tensor("Shape_2196:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 400, training accuracy nan
type  Tensor("Shape_2197:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 500, training accuracy nan
type  Tensor("Shape_2198:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 600, training accuracy nan
type  Tensor("Shape_2199:0", shape=TensorShape([Dimension(0)]), dtype=int32)
value  nan
step 700, training accuracy nan
type  Tensor("Shape_2200:0", sha

In [326]:

  
    
# test_xs,test_ys = testingDataLoader(category,my_testing)
print sess.run(accuracy, feed_dict={x: test_xs, y_: test_ys})


nan
